This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [1]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [2]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = 'train'  # e.g. './muffin-vs-chihuahua/train'
test_dir = 'test'    # e.g. './muffin-vs-chihuahua/test'

In [3]:
# IMAGE PARAMETERS
# Increased input size for transfer learning
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [4]:
# DATA PREPROCESSING & AUGMENTATION
# Enhanced data augmentation pipeline
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,  # Added zoom augmentation
    brightness_range=[0.8, 1.2],  # Added brightness augmentation
    shear_range=0.2,  # Added shear augmentation
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [5]:
# TRANSFER LEARNING WITH MOBILENETV2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import regularizers

base_model = MobileNetV2(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False  # Freeze base model

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [6]:
# Configure the model optimizers, loss function, and metrics
# Updated to use a fixed learning rate for compatibility with ReduceLROnPlateau
from tensorflow.keras.callbacks import ReduceLROnPlateau

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Fixed learning rate

# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5,  # Reduce learning rate by a factor of 0.5
    patience=3,  # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6  # Lower bound on the learning rate
)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# TRAINING THE IMPROVED CNN WITH LEARNING RATE SCHEDULING
history = model.fit(
    train_generator,
    epochs=15,  # Slightly more epochs for improved model
    validation_data=val_generator,
    callbacks=[reduce_lr]  # Added learning rate scheduler
)

Epoch 1/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 161s 1s/step - accuracy: 0.9710 - loss: 0.2877 - val_accuracy: 0.9862 - val_loss: 0.2470 - learning_rate: 0.0010
Epoch 2/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 99s 835ms/step - accuracy: 0.9863 - loss: 0.2161 - val_accuracy: 0.9926 - val_loss: 0.1777 - learning_rate: 0.0010
Epoch 3/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 107s 900ms/step - accuracy: 0.9905 - loss: 0.1711 - val_accuracy: 0.9862 - val_loss: 0.1706 - learning_rate: 0.0010
Epoch 4/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 99s 834ms/step - accuracy: 0.9855 - loss: 0.1580 - val_accuracy: 0.9820 - val_loss: 0.1700 - learning_rate: 0.0010
Epoch 5/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 95s 795ms/step - accuracy: 0.9916 - loss: 0.1323 - val_accuracy: 0.9905 - val_loss: 0.1311 - learning_rate: 0.0010
Epoch 6/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 1076s 9s/step - accuracy: 0.9921 - loss: 0.1147 - val_accuracy: 0.9894 - val_loss: 0.1143 - learning_rate: 0.0010
Epoch 7/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.9886 

In [8]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.9932 - loss: 0.0663
Test Accuracy: 0.9932432174682617


In [9]:
# SAVE THE MODEL
model.save('exercise_6_trained_model_improved.h5')

In [10]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prob = float(model.predict(img_array, verbose=0)[0,0])
    if prob >= 0.5:
        label = "Chihuahua"
        confidence = prob
    else:
        label = "Muffin"
        confidence = 1 - prob
    print(f"Prediction: {label} (confidence: {confidence:.2%})")


In [11]:
# Example usage:
predict_image("test/muffin/img_0_0.jpg")
predict_image("test/chihuahua/img_0_5.jpg")

Prediction: Chihuahua (confidence: 100.00%)


Prediction: Muffin (confidence: 99.91%)
